# Volumetric Benchmarking of Floquet Codes

In [1]:

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
from collections import defaultdict
from typing import Optional, List
import math
import sinter
import matplotlib
import rsmf
from main.codes.tic_tac_toe.gauge.GaugeFloquetColourCode import GaugeFloquetColourCode
from main.codes.tic_tac_toe.gauge.GaugeHoneycombCode import GaugeHoneycombCode
formatter : plt.Figure = rsmf.setup(r"\documentclass[a4paper,11pt,noarxiv]{quantumarticle}")


# Functions used to process the data

In [2]:
def get_sinter_fit(log_ps, sqrt_qs, target_x, stat):
    if len(log_ps) < 2:
        print('error, less than 2 points')
        return None
    
    slope_fit = sinter.fit_line_slope(
        xs=log_ps,
        ys=sqrt_qs,
        max_extra_squared_error=1,
    )
    if slope_fit.best >= 0:
        return None
    
    if slope_fit.high >= 0:
        # Slope is going the wrong way! Definitely over threshold.
        print('error, slope is going the wrong way')
        return None

    fit = sinter.fit_line_y_at_x(
        xs=log_ps,
        ys=sqrt_qs,
        target_x=target_x,
        max_extra_squared_error=1,  
    )
    return (fit)


def extrapolate_footprint_achieving_error_rate(
        group: List[sinter.TaskStats],
        target_p: float,
) -> Optional[sinter.Fit]:
    """Taken from Craig Gidney's code.
    
    Args:
        group: A list of TaskStats objects.
        target_p: The target probability of failure, for teraquop use 1e-12.

    Returns:
        A Fit object representing the footprint that would achieve the target
        probability of failure, or None if the data was insufficient.
    """
    assert len({stat.json_metadata['per'] for stat in group}) == 1
    sqrt_qs = []
    log_ps = []
    for stat in group:
        if stat.shots:
            p_shot = stat.errors / stat.shots
            if 0 < p_shot < 0.5:

                p_unit = p_shot
                sqrt_qs.append(math.sqrt(stat.json_metadata['distance']**2)) # huh why am is squaring and then taking the square root?
                log_ps.append(math.log(p_unit))
            

    return get_sinter_fit(log_ps, sqrt_qs, math.log(target_p), stat)

def low_error_for_multiplication(values, max_values):
    relative_error = 0
    abs_value = 1
    for value, max_value in zip(values, max_values):
        relative_error += (abs(max_value-value)/value)**2
        abs_value *= value
    
    relative_error = math.sqrt(relative_error)
    return(abs_value*(1-relative_error))

def high_error_for_multiplication(values, max_values):
    relative_error = 0
    abs_value = 1
    for value, max_value in zip(values, max_values):
        relative_error += (abs(max_value-value)/value)**2
        abs_value *= value

    relative_error = math.sqrt(relative_error)

    return(abs_value*(1+relative_error))


# Functions used to generate plots

In [3]:

from typing import Tuple


def float_to_color_shade(value: int, color_map: matplotlib.colors.LinearSegmentedColormap, vmin=200, vmax=3000):
    """
    Convert a float between 1 and 1000 to a shade of blue.

    Parameters
    ----------
    value : float
        The float value to convert.
    vmin : float
        The minimum value of the range. Default is 1.
    vmax : float
        The maximum value of the range. Default is 1000.

    Returns
    -------
    color : tuple
        The RGBA color corresponding to the input value.
    """
    # Normalize the value to the range [0, 1]
    norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
    normalized_value = norm(value)

    # Get the color corresponding to the normalized value
    color : Tuple[int,int,int,int]= color_map(normalized_value)
    return color

# Data processing

### Import and filter data

In [4]:

stats_memory : List[sinter.TaskStats] = sinter.stats_from_csv_files('./out/d_d_d_memory_experiment_0.001.csv')
stats_stability : List[sinter.TaskStats] = sinter.stats_from_csv_files('./out/new_d_d_d_stability_experiment_0.001.csv')
def filter_stats(stats, metadeta_entries: dict() = None, decoder = None):
    for metadeta_key, metadeta_value in metadeta_entries.items():
        stats = [stat for stat in stats if stat.json_metadata[metadeta_key] == metadeta_value]
    if decoder:
        stats = [stat for stat in stats if stat.decoder == decoder]
    return stats

stats_0001_memory_x_pymatching = filter_stats(stats_memory, metadeta_entries = {'per': 0.001, 'logical_observable': 'memory_x'}, decoder = 'pymatching')
stats_0001_memory_z_pymatching = filter_stats(stats_memory, metadeta_entries = {'per': 0.001, 'logical_observable': 'memory_z'}, decoder = 'pymatching')
stats_0001_memory_xz_pymatching = stats_0001_memory_x_pymatching + stats_0001_memory_z_pymatching

stats_0001_memory_x_beliefmatching = filter_stats(stats_memory, metadeta_entries = {'per': 0.001, 'logical_observable': 'memory_x'}, decoder = 'beliefmatching')
stats_0001_memory_z_beliefmatching = filter_stats(stats_memory, metadeta_entries = {'per': 0.001, 'logical_observable': 'memory_z'}, decoder = 'beliefmatching')
stats_0001_memory_xz_beliefmatching = stats_0001_memory_x_beliefmatching + stats_0001_memory_z_beliefmatching

stats_0001_stability_x_pymatching = filter_stats(stats_stability, metadeta_entries = {'per': 0.001, 'logical_observable': 'stability_x'}, decoder = 'pymatching')
stats_0001_stability_z_pymatching = filter_stats(stats_stability, metadeta_entries = {'per': 0.001, 'logical_observable': 'stability_z'}, decoder = 'pymatching')
stats_0001_stability_xz_pymatching = stats_0001_stability_x_pymatching + stats_0001_stability_z_pymatching

stats_0001_stability_x_beliefmatching = filter_stats(stats_stability, metadeta_entries = {'per': 0.001, 'logical_observable': 'stability_x'}, decoder = 'beliefmatching')
stats_0001_stability_z_beliefmatching = filter_stats(stats_stability, metadeta_entries = {'per': 0.001, 'logical_observable': 'stability_z'}, decoder = 'beliefmatching')
stats_0001_stability_xz_beliefmatching = stats_0001_stability_x_beliefmatching + stats_0001_stability_z_beliefmatching


### Calculate x-side and z-side

In [5]:
def calc_teraquop_patch_diameter(stats):
    noise_model_groups = sinter.group_by(stats, key = lambda stat: str(stat.json_metadata['px'])+ ',' + str(stat.json_metadata['py']) + ',' + str(stat.json_metadata['pz']) + ',' + str(stat.json_metadata['pm']))
    noise_model_footprints = defaultdict()

    for noise_model_key, noise_model_group in noise_model_groups.items():

        code_name_groups = sinter.group_by(noise_model_group, key = lambda stat: (stat.json_metadata['code_name'], stat.json_metadata['gf_0'], stat.json_metadata['gf_1'], stat.json_metadata['gf_2']))
        noise_model_footprints[noise_model_key] = defaultdict()
        for code_name_key, code_name_group in code_name_groups.items(): 
            noise_model_footprints[noise_model_key][code_name_key] = extrapolate_footprint_achieving_error_rate(
                    code_name_group,
                    target_p=10**(-12))
    return(noise_model_footprints)


In [6]:
x_patch_diameters = dict()
z_patch_diameters = dict()
x_patch_diameters["pymatching"] = calc_teraquop_patch_diameter(stats_0001_memory_x_pymatching)
z_patch_diameters["pymatching"] = calc_teraquop_patch_diameter(stats_0001_memory_z_pymatching)

In [7]:
x_patch_diameters["beliefmatching"] = calc_teraquop_patch_diameter(stats_0001_memory_x_beliefmatching)
z_patch_diameters["beliefmatching"] = calc_teraquop_patch_diameter(stats_0001_memory_z_beliefmatching)

### Calculate number of qubits

In [8]:
def calc_number_of_qubits(x_patch_diameters, z_patch_diamters):
    number_of_qubits = dict()
    for noise_model, diameters in x_patch_diameters.items():
        number_of_qubits[noise_model] = dict()
        for code_name, diameter in diameters.items():
            if z_patch_diamters[noise_model][code_name] and diameter:

                low_fit = 3*low_error_for_multiplication([diameter.best, z_patch_diamters[noise_model][code_name].best], [diameter.low, z_patch_diamters[noise_model][code_name].low])
                high_fit = 3*high_error_for_multiplication([diameter.best, z_patch_diamters[noise_model][code_name].best], [diameter.high, z_patch_diamters[noise_model][code_name].high])
                number_of_qubits[noise_model][code_name] = sinter.Fit(low=low_fit,
                                                                    best=3 * diameter.best * z_patch_diamters[noise_model][code_name].best,
                                                                        high=high_fit)
            else:
                print('error, no diameter', code_name, noise_model)
    return(number_of_qubits)

In [9]:
number_of_qubits = dict()
number_of_qubits["pymatching"] = calc_number_of_qubits(x_patch_diameters["pymatching"], z_patch_diameters["pymatching"])

In [10]:
number_of_qubits["beliefmatching"] = calc_number_of_qubits(x_patch_diameters["beliefmatching"], z_patch_diameters["beliefmatching"])

error, no diameter ('GaugeFloquetColourCode', 3, 3, 0) 1.0,1.0,4.0,1.0
error, no diameter ('GaugeFloquetColourCode', 1, 3, 0) 1.0,1.0,8.0,1.0
error, no diameter ('GaugeFloquetColourCode', 3, 3, 0) 1.0,1.0,8.0,4.0


### Calculate hight

In [11]:
def distance_to_hight(distance_fit: sinter.Fit, code_string: str, letter):
    if code_string[0] == "GaugeFloquetColourCode":
        code = GaugeFloquetColourCode(4, [code_string[1], code_string[2]])
    elif code_string[0] == "GaugeHoneycombCode":
        code = GaugeHoneycombCode(4, [code_string[1], code_string[2], code_string[3]])
    hight_low = code.get_graphlike_timelike_distance(math.ceil(distance_fit.low), letter, 'phenomenological_noise')
    hight_best = code.get_graphlike_timelike_distance(math.ceil(distance_fit.best), letter, 'phenomenological_noise')
    hight_high = code.get_graphlike_timelike_distance(math.ceil(distance_fit.high), letter, 'phenomenological_noise')
    return(sinter.Fit(hight_low, hight_best, hight_high))


def calc_hight(stats, letter):
    noise_model_groups = sinter.group_by(stats, key = lambda stat: str(stat.json_metadata['px'])+ ',' + str(stat.json_metadata['py']) + ',' + str(stat.json_metadata['pz']) + ',' + str(stat.json_metadata['pm']))
    noise_model_footprints = defaultdict()
    noise_model_hights = defaultdict()

    for noise_model_key, noise_model_group in noise_model_groups.items():

        code_name_groups = sinter.group_by(noise_model_group, key = lambda stat: (stat.json_metadata['code_name'], stat.json_metadata['gf_0'], stat.json_metadata['gf_1'], stat.json_metadata['gf_2']))
        noise_model_footprints[noise_model_key] = defaultdict()
        noise_model_hights[noise_model_key] = defaultdict()
        for code_name_key, code_name_group in code_name_groups.items(): 
            noise_model_footprints[noise_model_key][code_name_key] = extrapolate_footprint_achieving_error_rate(
                    code_name_group,
                    target_p=10**(-12))
            if noise_model_footprints[noise_model_key][code_name_key] != None:
                noise_model_hights[noise_model_key][code_name_key] = distance_to_hight(noise_model_footprints[noise_model_key][code_name_key], code_name_key, letter)
    return(noise_model_hights)

In [12]:
x_hights_stability=dict()
x_hights_stability["pymatching"] = calc_hight(stats_0001_stability_x_pymatching, 'X')
x_hights_stability["beliefmatching"] = calc_hight(stats_0001_stability_x_beliefmatching, 'X')


error, slope is going the wrong way


In [13]:
z_hights_stability=dict()

z_hights_stability["pymatching"] = calc_hight(stats_0001_stability_z_pymatching, 'Z')
z_hights_stability["beliefmatching"] = calc_hight(stats_0001_stability_z_beliefmatching, 'Z')


error, less than 2 points
error, less than 2 points


In [14]:
def calc_xz_hight(x_hights_stability, z_hights_stability):
    xz_hight = dict()
    for noise_model, x_hights_stability in x_hights_stability.items():
        xz_hight[noise_model] = dict()
        for code_name, x_hight in x_hights_stability.items():

            if code_name in z_hights_stability[noise_model]:
                z_hight = z_hights_stability[noise_model][code_name]
                xz_hight[noise_model][code_name] = sinter.Fit((x_hight.low + z_hight.low)/2, (x_hight.best + z_hight.best)/2, (x_hight.high + z_hight.high)/2)
            else:
                z_hight = None
                print('error, no hight', code_name, noise_model)
           
                

    return(xz_hight)



In [15]:
xz_hight = dict()
xz_hight["pymatching"] = calc_xz_hight(x_hights_stability["pymatching"], z_hights_stability["pymatching"])


In [16]:
xz_hight["beliefmatching"] = calc_xz_hight(x_hights_stability["beliefmatching"], z_hights_stability["beliefmatching"])

error, no hight ('GaugeFloquetColourCode', 1, 1, 0) 1.0,1.0,16.0,1.0
error, no hight ('GaugeFloquetColourCode', 2, 1, 0) 1.0,1.0,16.0,1.0


In [17]:
def calc_volumes(number_of_qubits, xz_hight):
    volumes = dict()
    for noise_model, code_name in number_of_qubits.items():
        volumes[noise_model] = dict()
        for code_name, number_of_qubit in code_name.items():
            if code_name in xz_hight[noise_model] and xz_hight[noise_model][code_name] != None:
                low_fit = low_error_for_multiplication([number_of_qubit.best, xz_hight[noise_model][code_name].best], [number_of_qubit.low, xz_hight[noise_model][code_name].low])
                high_fit = high_error_for_multiplication([number_of_qubit.best, xz_hight[noise_model][code_name].best], [number_of_qubit.high, xz_hight[noise_model][code_name].high])
                volumes[noise_model][code_name] = sinter.Fit(low=low_fit,
                        best=number_of_qubit.best * xz_hight[noise_model][code_name].best,
                        high=high_fit)

    return(volumes)


In [18]:
volumes = dict()
volumes["pymatching"] = calc_volumes(number_of_qubits["pymatching"], xz_hight["pymatching"])
volumes["beliefmatching"] = calc_volumes(number_of_qubits["beliefmatching"], xz_hight["beliefmatching"])

### Find best footprints

In [19]:
def get_best_volume(volumes_at_noise_model):
    best_volume_val = None
    best_volume_name = None
    for code_name, volumes in volumes_at_noise_model.items():
        if volumes.best is not None and (best_volume_val is None or volumes.best < best_volume_val):
            best_volume_val = volumes.best
            best_volume_name = code_name
    return best_volume_name, best_volume_val

def get_best_volumes(volumes):
    best_codes = dict()
    for noise_model in volumes.keys():
        best_codes[noise_model] = get_best_volume(volumes[noise_model])
    return(best_codes)

In [20]:
best_codes = dict()
best_codes['pymatching'] = get_best_volumes(volumes["pymatching"])
best_codes['beliefmatching'] = get_best_volumes(volumes["beliefmatching"])

### Functions for generating plots


In [21]:
def plot_best_footprints(best_codes: dict, ax: plt.Axes, vmin, vmax):

    x_y_to_color = dict()
    x_values = list()
    y_values = list()
    for noise_model, best_code in best_codes.items():
        x_val = float(noise_model.split(',')[-2])
        y_val = float(noise_model.split(',')[-1])
    
        if best_code[0][0] == 'GaugeHoneycombCode':
            x_y_to_color[(x_val, y_val)] = float_to_color_shade(best_code[1], plt.cm.Greens,vmin,vmax)
        elif best_code[0][0] == 'GaugeFloquetColourCode':
            x_y_to_color[(x_val, y_val)] = float_to_color_shade(best_code[1], plt.cm.Reds,vmin,vmax)    

        if x_val not in x_values:
            x_values.append(x_val)
        if y_val not in y_values:
            y_values.append(y_val)


    x_values.sort()
    y_values.sort()

    color_matrix = [[0 for i in range(len(y_values))] for j in range(len(x_values))]
    for x_val in x_values:
        for y_val in y_values:
            if (x_val, y_val) in x_y_to_color:
                color_matrix[y_values.index(y_val)][x_values.index(x_val)] = x_y_to_color[(x_val, y_val)]

    ax.imshow(color_matrix, origin = 'lower')
    ax.set_xticks(np.arange(len(x_values)), labels=x_values)
    ax.set_yticks(np.arange(len(y_values)), labels=y_values)
    ax.set_xlabel('Z error bias')
    ax.set_ylabel('Measurement error bias',)
    
def format_code_label(code_name):
    code_name_to_label = {
        'GaugeHoneycombCode': 'HCC',
        'GaugeFloquetColourCode': 'FCC'
    }
    if code_name_to_label.get(code_name[0], code_name) == 'HCC':
        return(f"$X^{code_name[1:][0]}Y^{code_name[1:][1]}Z^{code_name[1:][2]}$")
    else:
        return(f"$X^{code_name[1:][0]}Z^{code_name[1:][1]}$")



def plot_footprints(footprints,error_model, ax=plt, top_n=10, vmin=10,vmax=80, y_label = 'Patch diameter'):

    sorted_items = sorted(footprints[error_model].items(), key=lambda item: item[1].best if item[1] is not None else float('inf'))
    code_name_to_color = {'GaugeFloquetColourCode': plt.cm.Reds, 'GaugeHoneycombCode': plt.cm.Greens}
    for code_name, footprint in sorted_items[:top_n]:
        formatted_label = format_code_label(code_name)            
        bars = ax.bar(str(code_name), float(footprint.best), color = float_to_color_shade(footprint.best, code_name_to_color.get(code_name[0], 'black'), vmin=vmin, vmax=vmax))
        ax.errorbar(str(code_name), float(footprint.best), yerr=[[float(footprint.best - footprint.low)], [float(footprint.high - footprint.best)],], fmt='o', color='black', ecolor='black', elinewidth=1, capsize=3)
        for bar in bars:
            height = 1.03*footprint.high
            ax.text(bar.get_x() + bar.get_width() / 2.0, height, formatted_label, ha='center', va='bottom', rotation=90)
    ax.set_ylim(0, footprint.high + footprint.best)

#    ax.set_xlabel('Codes')
    ax.set_ylabel(y_label)
    ax.set_xticks([]) 
 
def draw_corner_bar_plots(footprints, fig, axd, n_points, vmin, vmax, ylabel, ymax):
    plot_footprints(footprints, '1.0,1.0,1.0,16.0', axd['A'],n_points, vmin=vmin, vmax=vmax, y_label=ylabel) 
    plot_footprints(footprints, '1.0,1.0,16.0,16.0', axd['C'],n_points, vmin=vmin, vmax=vmax, y_label=ylabel)

    plot_footprints(footprints, '1.0,1.0,1.0,1.0', axd['D'],n_points, vmin=vmin, vmax=vmax, y_label=ylabel) 
    plot_footprints(footprints, '1.0,1.0,16.0,1.0', axd['E'], n_points, vmin=vmin, vmax=vmax, y_label=ylabel)

    fig.tight_layout()

    axd['A'].set_ylim(0, ymax)
    axd['C'].sharey(axd['A'])
    axd['D'].sharey(axd['A'])
    axd['E'].sharey(axd['A'])

# Generate plots

In [22]:
def create_overview_plot(decoder: str,vmin: int, vmax: int, ymax: int):

    layout = [['A',   'C'],
            [ 'B',   'B',],
            [ 'B', 'B'],
            ['D', 'E']]
    fig : plt.Figure = formatter.figure(width_ratio=2, aspect_ratio=1.5)
    axd = fig.subplot_mosaic(layout, height_ratios=[1, 0.5, 0.5, 1],)
    fig.tight_layout()


    axd['A'].set_title('Volumes of the 10 best codes at \n Measurement bias 16, \n Z error bias 1')
    axd['C'].set_title('Volumes of the 10 best codes at \n Measurement bias 16, \n Z error bias 16')
    axd['D'].set_title('Volumes of the 10 best codes at \n Measurment bias 1 \n Z error bias 1')
    axd['E'].set_title('Volumes of the 10 best codes at \n Measurement bias 1 \n Z error bias 16')
    axd['B'].set_title('Volumes of the best code at each bias')

    plot_best_footprints(best_codes[decoder], axd['B'], vmin, vmax)
    draw_corner_bar_plots(volumes[decoder], fig, axd, 10, vmin, vmax, r"qubits $\times$ rounds", ymax)


    cb1=plt.colorbar(mappable=plt.cm.ScalarMappable(norm=mcolors.Normalize(vmin=vmin, vmax=vmax), cmap=plt.cm.Greens), ax=axd['B'], label='volume of $X^a Y^b Z^c$  DCCC')
    cb2=plt.colorbar(mappable=plt.cm.ScalarMappable(norm=mcolors.Normalize(vmin=vmin, vmax=vmax), cmap=plt.cm.Reds), ax=axd['B'], label='volume of $X^a Z^b$ DCCC')
    fig.tight_layout()
    l, b, w, h = axd['B'].get_position().bounds
    axd['B'].set_position([l, b+0.05, w, h])
    l, b, w, h = cb1.ax.get_position().bounds
    cb1.ax.set_position([l, b+0.05, w, h])
    l, b, w, h = cb2.ax.get_position().bounds
    cb2.ax.set_position([l, b+0.05, w, h])
    fig.savefig(f'spacetime_volume_heatmap_{decoder}.pdf', bbox_inches='tight')
    plt.close(fig)


In [23]:
create_overview_plot('pymatching', 10e2, 10e4, 3e5)
create_overview_plot('beliefmatching', 10e2, 5e4, 2e5)

In [24]:
def get_hcc_fcc_footprints(footprints, error_model, decoder):
    footprints_all_codes = [diameter for key,diameter in footprints[decoder].items() if key == error_model]
    footprints_hcc_fcc = {key: diameter for key,diameter in footprints_all_codes[0].items() if (key == ('GaugeFloquetColourCode', 1, 1, 0) or key == ('GaugeHoneycombCode', 1, 1, 1))}
    return footprints_hcc_fcc

def plot_all_footprints_fcc_hcc(footprints_list, error_model, ax=plt, x_labels=[], vmin=10,vmax=80, y_label = 'Patch diameter'):
    pos = 0
    code_name_to_color = {'GaugeFloquetColourCode': 'Red', 'GaugeHoneycombCode': 'Green'}

    for footprints in footprints_list:
        footprints_hcc_fcc_pymatching = get_hcc_fcc_footprints(footprints, error_model, 'pymatching')
        footprints_hcc_fcc_beliefmatching = get_hcc_fcc_footprints(footprints, error_model, 'beliefmatching')


        for code_name, footprint in footprints_hcc_fcc_pymatching.items():
            ax : plt.axes
          
            bars = ax.bar(pos, float(footprint.best), color=code_name_to_color.get(code_name[0], 'black'), hatch='x', edgecolor='black')
            ax.errorbar(pos, float(footprint.best), yerr=[[float(footprint.best - footprint.low)], [float(footprint.high - footprint.best)],], fmt='o', color='black', ecolor='black', elinewidth=1, capsize=3)

            pos += 1
                 
            bm_code_footprint = footprints_hcc_fcc_beliefmatching.get(code_name)

            bars = ax.bar(pos, float(bm_code_footprint.best), color=code_name_to_color.get(code_name[0], 'black'), edgecolor='black')
            ax.errorbar(pos, float(bm_code_footprint.best), yerr=[[float(bm_code_footprint.best - bm_code_footprint.low)], [float(bm_code_footprint.high - bm_code_footprint.best)],], fmt='o', color='black', ecolor='black', elinewidth=1, capsize=3)
            pos += 1.2
        pos += 1

    formatter = matplotlib.ticker.ScalarFormatter(useMathText=True)
    formatter.fontsize = 11
    formatter.set_scientific(True)
    formatter.set_powerlimits((-1, 1))
    ax.yaxis.set_major_formatter(formatter)
    ax.set_ylabel(y_label)
    ax.set_xticks(x_labels) 

In [35]:
fig = formatter.figure(wide=True)
import matplotlib.patches as mpatches

# create 3x1 subfigs
subfigs = fig.subfigures(nrows=3, ncols=1)
#fig.tight_layout()

# create 1x3 subplots per subfig
axs = subfigs[0].subplots(nrows=1, ncols=2, gridspec_kw={'width_ratios': [3, 1]})
subfigs[0].suptitle(f'Z error bias = 1, Measurement bias = 1')
x_labels = ['x-side', 'z-side', 'x-height', 'z-height']
plot_all_footprints_fcc_hcc([x_patch_diameters, z_patch_diameters, x_hights_stability, z_hights_stability],'1.0,1.0,1.0,1.0', axs[0], y_label= 'length')
plot_all_footprints_fcc_hcc([volumes], '1.0,1.0,1.0,1.0', axs[1], y_label= 'volume')

#print("\n")
axs = subfigs[1].subplots(nrows=1, ncols=2, gridspec_kw={'width_ratios': [3, 1]})
subfigs[1].suptitle(f'Z error bias = 8, Measurement bias = 1')
plot_all_footprints_fcc_hcc([x_patch_diameters, z_patch_diameters, x_hights_stability],'1.0,1.0,8.0,1.0', axs[0], y_label= 'length')
plot_all_footprints_fcc_hcc([x_patch_diameters, z_patch_diameters, x_hights_stability, z_hights_stability],'1.0,1.0,8.0,1.0', axs[0], y_label= 'length')
plot_all_footprints_fcc_hcc([volumes], '1.0,1.0,8.0,1.0', axs[1], y_label= 'volume')
#axs[0].set_xticks([0.5,3.5,6.5,9.5], x_labels)

subfigs[2].suptitle(f'Z error bias = 1, Measurement bias = 8')
axs = subfigs[2].subplots(nrows=1, ncols=2, gridspec_kw={'width_ratios': [3, 1]})

plot_all_footprints_fcc_hcc([x_patch_diameters, z_patch_diameters, x_hights_stability, z_hights_stability],'1.0,1.0,1.0,16.0', axs[0], y_label= 'length')
plot_all_footprints_fcc_hcc([volumes], '1.0,1.0,1.0,16.0', axs[1], y_label= 'volume')
axs[0].set_xticks([0.5,3.5,6.5,9.5], x_labels)
red_patch_pm = mpatches.Patch(label='CSS DCCC pymatching', edgecolor='black', hatch='//', facecolor='red')
red_patch_bm = mpatches.Patch(label='CSS DCCC beliefmatching', edgecolor='black', facecolor='red')
green_patch_pm = mpatches.Patch(facecolor='green', label='XYZ DCCC pymatching', edgecolor='black', hatch='//')
green_patch_bm = mpatches.Patch(facecolor='green', label='XYZ DCCC beliefmatching', edgecolor='black')
axs[0].set_xticks([1.625,7,12.375,17.8], x_labels)
axs[0].legend(handles=[red_patch_pm, red_patch_bm, green_patch_pm, green_patch_bm], loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2)

#    ax.set_title(f'Plot title {col}')


fig.savefig('CSS_DCCC_XYZ_DCCC_comparison.pdf', bbox_inches='tight')
plt.close(fig)  # Close the figure to free up memory



{'pymatching': defaultdict(None, {'1.0,1.0,1.0,1.0': defaultdict(None, {('GaugeFloquetColourCode', 1, 1, 0): sinter.Fit(low=36.91032653502672, best=40.34538512877672, high=43.78044372252672), ('GaugeFloquetColourCode', 1, 2, 0): sinter.Fit(low=51.6326483456632, best=56.9008368222257, high=62.1690252987882), ('GaugeFloquetColourCode', 2, 2, 0): sinter.Fit(low=41.202117561055104, best=45.169280646992604, high=49.136443732930104), ('GaugeFloquetColourCode', 2, 1, 0): sinter.Fit(low=33.030116328178345, best=35.97918249028772, high=38.928248652397095), ('GaugeFloquetColourCode', 1, 3, 0): sinter.Fit(low=76.23206857252156, best=84.63258126783406, high=93.03309396314656), ('GaugeFloquetColourCode', 2, 3, 0): sinter.Fit(low=60.51602633948755, best=66.90677340980005, high=73.29752048011255), ('GaugeFloquetColourCode', 3, 1, 0): sinter.Fit(low=34.34746583995253, best=37.45784792003065, high=40.56823000010878), ('GaugeFloquetColourCode', 3, 2, 0): sinter.Fit(low=45.500816282140924, best=50.003013

In [26]:
print(volumes['1.0,1.0,1.0,16.0'][("GaugeFloquetColourCode",2,2,0)].best*10e-7)
print(volumes['1.0,1.0,1.0,16.0'][("GaugeFloquetColourCode",1,1,0)].best*10e-7)

sinter.collect()
#print(volumes_SI1000[("GaugeHoneycombCode",2,1,1)].best*10e-8)
#rint(volumes_SI1000[("GaugeFloquetColourCode",1,1,0)].best/volumes_SI1000[("GaugeHoneycombCode",2,1,1)].best)

KeyError: '1.0,1.0,1.0,16.0'

In [62]:
import numpy as np

# Data
categories = ['A']
values1 = [10]
values2 = [6]

# Bar width
bar_width = 0.4

# Positions of the bars on the x-axis
r1 = np.arange(len(categories))
r2 = [x - bar_width/2 for x in r1]
plt.figure()
# Create bars
plt.bar(r1, values1, color='b', width=bar_width, edgecolor='grey', label='Bigger Bar')
plt.bar(r2, values2, color='r', width=bar_width, edgecolor='grey', label='Smaller Bar')

# Add labels
plt.xlabel('Category', fontweight='bold')
plt.xticks([r for r in range(len(categories))], categories)
plt.ylabel('Values', fontweight='bold')

# Add legend
plt.legend()

# Show plot
plt.savefig('barplot.png')
